In [50]:
%%capture

from google.colab import drive, files
drive.mount('/content/drive/')
%cd /content/drive/My Drive/AI/NewsCrawl

In [51]:
import pandas as pd
import numpy as np
from glob import glob
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

# Load dữ liệu từ dataset

In [52]:
dataset = []
dataset.extend(glob('./vnexpress-dantri-merged/*.xlsx'))
dataset

['./vnexpress-dantri-merged/thoi-su.xlsx',
 './vnexpress-dantri-merged/the-gioi.xlsx',
 './vnexpress-dantri-merged/kinh-doanh.xlsx',
 './vnexpress-dantri-merged/the-thao.xlsx',
 './vnexpress-dantri-merged/giai-tri.xlsx',
 './vnexpress-dantri-merged/phap-luat.xlsx',
 './vnexpress-dantri-merged/suc-khoe.xlsx',
 './vnexpress-dantri-merged/giao-duc.xlsx',
 './vnexpress-dantri-merged/khoa-hoc-cong-nghe.xlsx',
 './vnexpress-dantri-merged/du-lich.xlsx',
 './vnexpress-dantri-merged/oto-xe-may.xlsx',
 './vnexpress-dantri-merged/train.xlsx',
 './vnexpress-dantri-merged/test.xlsx']

In [53]:
train_df = pd.DataFrame()
test_df = pd.DataFrame()
train_df = train_df.append(pd.read_excel('./vnexpress-dantri-merged/train.xlsx'), ignore_index=True)
test_df = test_df.append(pd.read_excel('./vnexpress-dantri-merged/test.xlsx'), ignore_index=True)
train_df.dropna(inplace = True) # Loại bỏ null
test_df.dropna(inplace = True) # Loại bỏ null

In [54]:
train_df.head()

,Unnamed: 0,id,title,content,label
0,0,2fdc17c5-3abc-4a32-b565-4f9a31b0b63f,Đề xuất xây đường kéo dài đại lộ Thăng Long,"Ngày 17/5, đại diện Bộ Giao thông Vận tải cho ...",0
1,1,4fa7b6b0-5969-4927-9392-d9981971d3e3,Đề xuất xây thêm bãi đỗ xe tại sân bay Nội Bài,Theo đề xuất của Cục Hàng không Việt Nam gửi B...,0
2,2,ef592a33-23e7-496e-a7bd-0cef5f4b3cd2,Đề xuất xóa 4 tổng cục thuộc Bộ Nông nghiệp,"Theo dự thảo nghị định quy định chức năng, nhi...",0
3,3,3a699019-f1a8-4d25-8427-1f1e62329521,Đề xuất xử lý hình sự hành vi thu gom sổ bảo h...,"Kiến nghị được Giám đốc Sở Lao động, Thương bi...",0
4,4,582a4158-78be-44a0-b7e9-9019abfbf1ad,Đêm hội trăng rằm của trẻ có hoàn cảnh khó khăn,Chương trình mở cửa miễn phí với nhiều tiết mụ...,0


In [55]:
test_df.head()

,Unnamed: 0,id,title,content,label
0,0,efde56c2-b0d4-4aef-91dd-874058c186ac,1.000 ha dừa Bến Tre nhiễm sâu đầu đen,"Sáng gần giữa tháng tư, ông Nguyễn Long Hổ (50...",0
1,1,26af923d-7550-4d59-a5cd-1ccfb2e988ff,1.000 túi thuốc hỗ trợ F0 điều trị tại nhà,"Sáng 17/9, 1.000 túi thuốc đóng gói sẵn được c...",0
2,2,9bc5be8f-460f-46e7-b418-3e3a3464e43e,1/4 số thực tập sinh Nhật Bản trở về Việt Nam ...,Cơ quan Hợp tác Quốc tế Nhật Bản (JICA) vừa cô...,0
3,3,adfad0b4-7924-483e-ac26-2e12de121ca0,"1,5 triệu người chưa nhận, TP HCM kéo dài hỗ t...","Chiều 22/10, Giám đốc Sở Lao động, Thương binh...",0
4,4,2e4ab952-2a50-41fc-a32b-f40c51495157,1.500 tỷ đồng xây mới cầu Trà Khúc 1,"Chiều 19/4, Chủ tịch UBND Quảng Ngãi Đặng Văn ...",0


In [56]:
labels = []
for label in train_df['label']:
    labels.append(label)


labels = list(set(labels))
print(f"Có tổng cộng {len(labels)} nhãn")
print(list(set(labels)))

Có tổng cộng 11 nhãn
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [57]:
id2label = {
    0: 'thời sự',
    1: 'thế giới',
    2: 'kinh doanh',
    3: 'thể thao',
    4: 'giải trí',
    5: 'pháp luật',
    6: 'sức khỏe',
    7: 'giáo dục',
    8: 'khoa học - công nghệ',
    9: 'du lịch - ẩm thực',
    10: 'oto xe máy'
}


label_names = [id2label[i] for i in range(len(labels))]
print(label_names)

['thời sự', 'thế giới', 'kinh doanh', 'thể thao', 'giải trí', 'pháp luật', 'sức khỏe', 'giáo dục', 'khoa học - công nghệ', 'du lịch - ẩm thực', 'oto xe máy']


# Các hàm hỗ trợ

In [58]:
def get_corpus_and_label(df):
    title = [title for title in df['title']]
    content = [content for content in df['content']]
    label = [label for label in df['label']]

    corpus = []
    for index in range(len(title)):
        # merge tiêu đề và nội dung bài viết
        text = title[index] + ' ' + content[index]
        corpus.append(text)
    
    return label, corpus

In [59]:
# Lấy nhãn và corpus từ dataset

train_label, train_corpus = get_corpus_and_label(train_df)
test_label, test_corpus = get_corpus_and_label(test_df)

In [60]:
# sns.set(font_scale=1.4)
# cm = confusion_matrix(y_test, y_pred) 
# cm_df = pd.DataFrame(cm,
#                      index = label_names, 
#                      columns = label_names)

# plt.figure(figsize=(10,8))
# sns.heatmap(cm_df, annot=True, fmt="d",annot_kws={"size": 18})
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.title('Countvector')
# plt.show()

# Train và Test

## Naive Bayes

### Countvector

In [61]:
# Train

cv = CountVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)

nb_classifier_1 = MultinomialNB()
nb_classifier_1.fit(x_train, y_train)

# Test

x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

y_pred = nb_classifier_1.predict(x_test)
nb1_acc = accuracy_score(y_test, y_pred)
print(nb1_acc)

0.8129090909090909


In [62]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.72      0.73      1000
           1       0.88      0.71      0.79      1000
           2       0.78      0.83      0.81      1000
           3       0.96      0.94      0.95      1000
           4       0.70      0.86      0.77      1000
           5       0.87      0.73      0.79      1000
           6       0.79      0.85      0.82      1000
           7       0.81      0.90      0.85      1000
           8       0.88      0.53      0.66      1000
           9       0.71      0.89      0.79      1000
          10       0.91      0.96      0.94      1000

    accuracy                           0.81     11000
   macro avg       0.82      0.81      0.81     11000
weighted avg       0.82      0.81      0.81     11000



### Tfidf 

In [63]:
# Train

cv = TfidfVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)

nb_classifier_2 = MultinomialNB()
nb_classifier_2.fit(x_train, y_train)

# Test

x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

y_pred = nb_classifier_2.predict(x_test)
nb2_acc = accuracy_score(y_test, y_pred)
print(nb2_acc)

0.8098181818181818


In [64]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.70      0.70      1000
           1       0.87      0.74      0.80      1000
           2       0.68      0.89      0.77      1000
           3       0.97      0.94      0.96      1000
           4       0.77      0.86      0.81      1000
           5       0.90      0.55      0.68      1000
           6       0.77      0.90      0.83      1000
           7       0.75      0.95      0.84      1000
           8       0.89      0.67      0.76      1000
           9       0.80      0.86      0.83      1000
          10       0.97      0.84      0.90      1000

    accuracy                           0.81     11000
   macro avg       0.82      0.81      0.81     11000
weighted avg       0.82      0.81      0.81     11000



## Logistic Regression

### Countvector

In [65]:
# Train

cv = CountVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)

lr_classifier_1 = LogisticRegression(multi_class='multinomial')
lr_classifier_1.fit(x_train, y_train)

# Test

x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

y_pred = lr_classifier_1.predict(x_test)
lr1_acc = accuracy_score(y_test, y_pred)
print(lr1_acc)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.8859090909090909


In [66]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.83      0.83      1000
           1       0.92      0.89      0.91      1000
           2       0.85      0.86      0.86      1000
           3       0.98      0.97      0.97      1000
           4       0.84      0.91      0.87      1000
           5       0.96      0.76      0.85      1000
           6       0.84      0.94      0.89      1000
           7       0.88      0.93      0.90      1000
           8       0.83      0.86      0.85      1000
           9       0.90      0.81      0.85      1000
          10       0.96      0.98      0.97      1000

    accuracy                           0.89     11000
   macro avg       0.89      0.89      0.89     11000
weighted avg       0.89      0.89      0.89     11000



### Tfidf

In [67]:
# Train

cv = TfidfVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)

lr_classifier_2 = LogisticRegression(multi_class='multinomial')
lr_classifier_2.fit(x_train, y_train)

# Test

x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

y_pred = lr_classifier_2.predict(x_test)
lr2_acc = accuracy_score(y_test, y_pred)
print(lr2_acc)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.9018181818181819


In [68]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.82      0.83      1000
           1       0.91      0.88      0.89      1000
           2       0.84      0.90      0.87      1000
           3       0.97      0.98      0.98      1000
           4       0.92      0.91      0.91      1000
           5       0.97      0.75      0.85      1000
           6       0.84      0.98      0.90      1000
           7       0.90      0.96      0.93      1000
           8       0.91      0.87      0.89      1000
           9       0.89      0.92      0.90      1000
          10       0.96      0.96      0.96      1000

    accuracy                           0.90     11000
   macro avg       0.90      0.90      0.90     11000
weighted avg       0.90      0.90      0.90     11000



## Decision Tree

### Counvector

In [69]:
# Train

cv = CountVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)

tree_classifier_1 = DecisionTreeClassifier()
tree_classifier_1.fit(x_train, y_train)

# Test

x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

y_pred = tree_classifier_1.predict(x_test)
tree1_acc = accuracy_score(y_test, y_pred)
print(tree1_acc)

0.7180909090909091


In [70]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.54      0.56      1000
           1       0.62      0.61      0.62      1000
           2       0.61      0.72      0.66      1000
           3       0.89      0.86      0.87      1000
           4       0.71      0.67      0.69      1000
           5       0.78      0.49      0.60      1000
           6       0.68      0.81      0.74      1000
           7       0.73      0.85      0.78      1000
           8       0.68      0.65      0.66      1000
           9       0.73      0.70      0.72      1000
          10       0.90      1.00      0.95      1000

    accuracy                           0.72     11000
   macro avg       0.72      0.72      0.71     11000
weighted avg       0.72      0.72      0.71     11000



### Tfidf

In [71]:
# Train

cv = TfidfVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)

tree_classifier_2 = DecisionTreeClassifier()
tree_classifier_2.fit(x_train, y_train)

# Test

x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

y_pred = tree_classifier_2.predict(x_test)
tree2_acc = accuracy_score(y_test, y_pred)
print(tree2_acc)

0.7162727272727273


In [72]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.54      0.55      1000
           1       0.64      0.60      0.62      1000
           2       0.61      0.69      0.65      1000
           3       0.88      0.85      0.86      1000
           4       0.69      0.69      0.69      1000
           5       0.80      0.46      0.58      1000
           6       0.67      0.82      0.74      1000
           7       0.73      0.83      0.78      1000
           8       0.69      0.66      0.68      1000
           9       0.74      0.73      0.73      1000
          10       0.92      1.00      0.96      1000

    accuracy                           0.72     11000
   macro avg       0.72      0.72      0.71     11000
weighted avg       0.72      0.72      0.71     11000



## SVM

### Countvector

In [73]:
# Train

cv = CountVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)

svm_classifier_1 = LinearSVC()
svm_classifier_1.fit(x_train, y_train)

# Test

x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

y_pred = svm_classifier_1.predict(x_test)
svm1_acc = accuracy_score(y_test, y_pred)
print(svm1_acc)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


0.885


In [74]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.81      0.79      1000
           1       0.90      0.88      0.89      1000
           2       0.83      0.86      0.85      1000
           3       0.97      0.97      0.97      1000
           4       0.90      0.90      0.90      1000
           5       0.94      0.75      0.83      1000
           6       0.84      0.95      0.89      1000
           7       0.88      0.94      0.91      1000
           8       0.88      0.83      0.86      1000
           9       0.88      0.85      0.86      1000
          10       0.95      1.00      0.98      1000

    accuracy                           0.89     11000
   macro avg       0.89      0.88      0.88     11000
weighted avg       0.89      0.89      0.88     11000



### Tfidf

In [75]:
# Train

cv = TfidfVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)

svm_classifier_2 = LinearSVC()
svm_classifier_2.fit(x_train, y_train)

# Test

x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

y_pred = svm_classifier_2.predict(x_test)
svm2_acc = accuracy_score(y_test, y_pred)
print(svm2_acc)

0.9157272727272727


In [76]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.83      0.85      1000
           1       0.93      0.91      0.92      1000
           2       0.87      0.91      0.89      1000
           3       0.98      0.99      0.98      1000
           4       0.93      0.91      0.92      1000
           5       0.98      0.79      0.88      1000
           6       0.85      0.97      0.91      1000
           7       0.90      0.96      0.93      1000
           8       0.92      0.89      0.91      1000
           9       0.91      0.91      0.91      1000
          10       0.96      0.99      0.98      1000

    accuracy                           0.92     11000
   macro avg       0.92      0.92      0.92     11000
weighted avg       0.92      0.92      0.92     11000



# TỔNG HỢP

In [77]:
board = pd.DataFrame(
    index = ['Naive Bayes', 'Logictic Regression', 'Decision Tree', 'SVM'],
    columns= ['Countvector', 'Tfidf'],
    data= [
              [nb1_acc,nb2_acc],
              [lr1_acc,lr2_acc],
              [tree1_acc, tree2_acc],
              [svm1_acc, svm2_acc]
          ]
)

In [78]:
board

,Countvector,Tfidf
Naive Bayes,0.812909,0.809818
Logictic Regression,0.885909,0.901818
Decision Tree,0.718091,0.716273
SVM,0.885000,0.915727


# Ensemble

In [89]:
from sklearn.ensemble import VotingClassifier

cv = TfidfVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)
x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)


voting_classifier = VotingClassifier( estimators=[
    ('nb1', nb_classifier_1),
    ('nb2', nb_classifier_2),
    # ('lr1', lr_classifier_1),
    ('lr2', lr_classifier_2),
    # ('svm1', svm_classifier_1),
    ('svm2', svm_classifier_2)
    ], voting='hard', weights=[1, 1, 2, 2])

voting_classifier.fit(x_train, y_train)


for clf in (nb_classifier_1,nb_classifier_2,lr_classifier_1, lr_classifier_2, svm_classifier_1, svm_classifier_2, voting_classifier):
    y_pred = clf.predict(x_test)  
    print(clf.__class__.__name__ , accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


MultinomialNB 0.801
MultinomialNB 0.8098181818181818
LogisticRegression 0.22736363636363635
LogisticRegression 0.9018181818181819
LinearSVC 0.6125454545454545
LinearSVC 0.9157272727272727
VotingClassifier 0.9022727272727272


In [90]:
from sklearn.ensemble import VotingClassifier

cv = TfidfVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)
x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)


voting_classifier = VotingClassifier( estimators=[
    ('nb1', nb_classifier_1),
    ('nb2', nb_classifier_2),
    # ('lr1', lr_classifier_1),
    ('lr2', lr_classifier_2),
    # ('svm1', svm_classifier_1),
    ('svm2', svm_classifier_2)
    ], voting='soft', weights=[1, 1, 2, 2], flatten_transform=True)

voting_classifier.fit(x_train, y_train)


for clf in (nb_classifier_1,nb_classifier_2,lr_classifier_1, lr_classifier_2, svm_classifier_1, svm_classifier_2, voting_classifier):
    y_pred = clf.predict(x_test)  
    print(clf.__class__.__name__ , accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


MultinomialNB 0.801
MultinomialNB 0.8098181818181818
LogisticRegression 0.22736363636363635
LogisticRegression 0.9018181818181819
LinearSVC 0.6125454545454545
LinearSVC 0.9157272727272727


AttributeError: ignored

In [85]:
news = 'Đối với nhiều trường, việc lùi giờ học này thực chất chỉ là “lùi giờ đóng cổng trường” thêm 15-25 phút, còn giờ học vẫn bắt đầu từ 7 giờ không đổi. Trước đây, dù 7 giờ mới bắt đầu tiết học đầu tiên nhưng nhiều trường quy định giờ đóng cổng là 6 giờ 45 hay có trường là 6 giờ 35. Với giờ đóng cổng như vậy, nỗi ám ảnh đi trễ luôn đè nặng tâm trí học sinh vì đến trễ giờ đóng cổng sẽ bị ghi tên, đi trễ ba lần sẽ bị hạ một bậc hạnh kiểm. Ngoài ảnh hưởng có hại cho tâm lý học sinh và phụ huynh, việc phải “chạy đua” cho kịp giờ đóng cổng như vậy cũng là một trong những lý do dẫn đến tai nạn giao thông. Đã có những vụ bị thương tích rất nặng như một học sinh lớp 10 bị xe khách đụng khi cách trường chỉ còn 50 mét hồi tháng trước. Trong mấy năm đưa con đi học, đặc biệt vào mùa mưa, không ít lần người viết bài này chứng kiến học sinh trượt chân té nhào khi chạy bộ hay té xe khi gấp rút chạy đến trường cho kịp giờ đóng cổng. Nhìn các em lấm lem bùn đất, trầy trụa chân tay thật xót xa. Sau khi có quy định mới, các trường đã lùi giờ đóng cổng đến 6 giờ 55, tức chỉ chậm thêm 10-20 phút nhưng lợi ích mang lại rất lớn. Cảnh tượng phụ huynh, học sinh phải đua xe bất đắc dĩ hầu như không còn nữa, tâm lý học sinh cũng thoải mái vì thoát được nỗi ám ảnh bị ghi tên, hạ hạnh kiểm. Việc thay đổi có lợi như thế này hầu như không ảnh hưởng đến thời khóa biểu vì giờ học vẫn như cũ. Tuy nhiên, vấn đề này trước đây mỗi khi được đưa ra thảo luận trên báo chí hay các cuộc họp của ngành giáo dục thì đều bị bàn ra với những lý do rất vĩ mô như “để hạn chế kẹt xe” hay “tránh gây ảnh hưởng đến kế hoạch học tập”… Thực tế cho thấy, việc thay đổi giờ đóng cổng trường như vậy lẽ ra cần áp dụng từ lâu, nhưng mãi đến khi chính Sở Giáo dục và Đào tạo TPHCM thay đổi cách nghĩ thì mới áp dụng được.'

id2label[clf.predict(cv.transform([news]).toarray())[0]]

'giáo dục'